# Gaussian Distributions Utilities

In [ ]:
#| hide
#| default_exp gaussian

## Normal Parameters

In [ ]:
#| export
from collections import namedtuple
from fastcore.basics import patch

### Normal

In [ ]:
#| export
ListNormal = namedtuple('ListNormal', ['mean', 'std'])

In [ ]:
#| export
Normal = namedtuple('Normal', ['mean', 'std'])

In [ ]:
#| export
@patch
def get_nth(self: ListNormal, n:int
           )->Normal:
    """Get the mean and cov for the nth Normal distribution in the list """
    return Normal(self.mean[n], self.std[n])

In [ ]:
#| export
@patch
def detach(self: ListNormal)->ListNormal:
    """Detach both mean and cov at once """
    return ListNormal(self.mean.detach(), self.std.detach())

In [ ]:
ListNormal(torch.rand(10), torch.rand(10)).get_nth(1)

Normal(mean=tensor(0.0450), std=tensor(0.0348))

### Multivariate Normal

In [ ]:
#| export
ListMNormal = namedtuple('ListNormal', ['mean', 'cov'])

In [ ]:
#| export
MNormal = namedtuple('Normal', ['mean', 'cov'])

In [ ]:
#| export
@patch
def get_nth(self: ListMNormal, n:int
           )->Normal:
    """Get the mean and cov for the nth Normal distribution in the list """
    return Normal(self.mean[n], self.cov[n])

In [ ]:
#| export
@patch
def detach(self: ListMNormal)->ListMNormal:
    """Detach both mean and cov at once """
    return ListMNormal(self.mean.detach(), self.cov.detach())

### Conditional Predictions

This add the supports for conditional predictions, which means that at the time (t) when we are making the predictions some of the variables have been actually observed. Since the model prediction is a normal distribution we can condition on the observed values and thus improve the predictions.

Therefore we need to compute the conditional distribution of a normal ^[https://cs.nyu.edu/~roweis/notes/gaussid.pdf eq, 5a, 5d]

$$ X = \left[\begin{array}{c} x \\ o \end{array} \right] $$

$$ p(X) = N\left(\left[ \begin{array}{c} \mu_x \\ \mu_o \end{array} \right], \left[\begin{array}{cc} \Sigma_{xx} & \Sigma_{xo} \\ \Sigma_{ox} & \Sigma_{oo} \end{array} \right]\right)$$

where $x$ is a vector of variable that need to predicted and $o$ is a vector of the variables that have been observed


then the conditional distribution is:

$$p(x|o) = N(\mu_x + \Sigma_{xo}\Sigma_{oo}^{-1}(o - \mu_o), \Sigma_{xx} - \Sigma_{xo}\Sigma_{oo}^{-1}\Sigma_{ox})$$

In [ ]:
#| export
import torch
from torch.distributions import MultivariateNormal
from torch.linalg import cholesky
from torch import cholesky_inverse
from torch import Tensor

from fastcore.test import *
from meteo_imp.utils import *

This is the direct implementation of the equations

In [ ]:
def _conditional_guassian_base(
                         μ: Tensor, # mean with shape `[n_vars]`
                         Σ: Tensor, # cov with shape `[n_vars, n_vars] `
                         obs: Tensor, # Observations with shape `[n_vars]`
                         idx: Tensor # Boolean tensor specifying for each variable is observed (True) or not (False). Shape `[n_vars]`
                        ) -> ListNormal: # Distribution conditioned on observations
    μ_x = μ[~idx]
    μ_o = μ[idx]
    
    Σ_xx = Σ[~idx,:][:, ~idx]
    Σ_xo = Σ[~idx,:][:, idx]
    Σ_ox = Σ[idx,:][:, ~idx]
    Σ_oo = Σ[idx,:][:, idx]
    
    Σ_oo_inv = torch.linalg.inv(Σ_oo)
    
    mean = μ_x + Σ_xo@Σ_oo_inv@(obs - μ_o)
    cov = Σ_xx - Σ_xo@Σ_oo_inv@Σ_ox
    
    return ListNormal(mean, cov)
    

 faster version

In [ ]:
#| export
def conditional_guassian(
                         μ: Tensor, # mean with shape `[n_vars]`
                         Σ: Tensor, # cov with shape `[n_vars, n_vars] `
                         obs: Tensor, # Observations with shape `[n_obs]`, where `n_obs = sum(idx)`
                         idx: Tensor # Boolean tensor specifying for each variable is observed (True) or not (False). Shape `[n_vars]`
                        ) -> ListNormal: # Distribution conditioned on observations. shape `[n_vars - n_obs]`
    assert μ.shape[0] == idx.shape[0]
    assert obs.shape[0] == sum(idx)
    
    μ_x = μ[~idx]
    μ_o = μ[idx]
    
    Σ_xx = Σ[~idx,:][:, ~idx]
    Σ_xo = Σ[~idx,:][:, idx]
    Σ_ox = Σ[idx,:][:, ~idx]
    Σ_oo = Σ[idx,:][:, idx]
    
    Σ_oo_inv = cholesky_inverse(cholesky(Σ_oo))
    
    
    mean = μ_x + Σ_xo@Σ_oo_inv@(obs - μ_o)
    cov = Σ_xx - Σ_xo@Σ_oo_inv@Σ_ox
    
    return ListNormal(mean, cov)
    

In [ ]:
# example distribution with only 2 variables
μ = torch.tensor([.5, 1.])
Σ = torch.tensor([[1., .5], [.5 ,1.]])


idx = torch.tensor([True, False]) # second variable is the observed one

obs = torch.tensor(5.) # value of second variable

gauss_cond = conditional_guassian(μ, Σ, obs, idx)

# hardcoded values to test that the code is working, see also for alternative implementation https://python.quantecon.org/multivariate_normal.html
test_close(3.25, gauss_cond.mean.item())
test_close(.75, gauss_cond.cov.item())

## Improvements

Use `cholesky` decomposition and `cholesky_solve` to improve performance of matrix inversion

see the [Probabilist machine learning course from uni Tübigen](https://uni-tuebingen.de/en/180804), specifically the code from the [Gaussian Regression Notebook](https://uni-tuebingen.de/fileadmin/Uni_Tuebingen/Fakultaeten/MatNat/Fachbereiche/Informatik/Lehrstuehle/MethMaschLern/Probabilistic_ML/Notebook_Vorlesung_7___9/Gaussian_Linear_Regression.ipynb) for details

In [ ]:
#| export
def to_posdef(A):
    return A @ A.mT + 1e-3

In [ ]:
n_var = 5
mean = torch.rand(n_var)
cov = to_posdef(torch.rand(n_var, n_var))
dist = MultivariateNormal(mean, cov)
idx = torch.rand(n_var) > .5
obs = torch.rand(n_var)[idx]

In [ ]:
torch.linalg.inv(cov) 

tensor([[ 2.4536e+00, -1.0228e-01, -5.5039e+00,  3.1706e+00,  1.2639e+00],
        [-1.0228e-01,  2.1123e+00, -1.4500e+00,  8.7551e-02,  3.5327e-01],
        [-5.5039e+00, -1.4500e+00,  1.5776e+02, -1.4320e+02, -7.4003e+01],
        [ 3.1706e+00,  8.7559e-02, -1.4320e+02,  1.3369e+02,  6.7287e+01],
        [ 1.2640e+00,  3.5327e-01, -7.4003e+01,  6.7287e+01,  3.7809e+01]])

In [ ]:
test_close(torch.linalg.inv(cov), cholesky_inverse(torch.linalg.cholesky(cov)), eps=2e-3)

In [ ]:
A = to_posdef(torch.rand(2000, 2000)) 

In [ ]:
%timeit torch.linalg.inv(A)

70.8 ms ± 6.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit cholesky_inverse(torch.linalg.cholesky(A))

53.5 ms ± 4.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


The second version is a bit faster

In [ ]:
test_close(conditional_guassian(dist, obs, idx).mean, _conditional_guassian_base(dist, obs, idx).mean)

In [ ]:
B = to_posdef(torch.rand(n_var, n_var))

In [ ]:
B @ torch.inverse(cov)

In [ ]:
torch.cholesky_solve(cholesky(cov), B)

## Export

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()